# Appl.ai : Automated Cover Letter Generation with Multi-Agent Feedback System

## Introduction 

In today's competitive job market, crafting personalized cover letters that accurately represent a candidate's qualifications while meeting specific job requirements is crucial yet time-consuming. This project implements an intelligent multi-agent system that automates and optimizes the cover letter generation process using AI.
Our system employs three specialized agents: a Resume Reviewer for extracting and structuring key information from CVs, a Cover Letter Writer for generating initial drafts, and a Quality Improvement Agent for ensuring accuracy and relevance. The system features an iterative feedback loop that validates content against the original resume, eliminates hallucinations, and maintains professional standards. Using OpenAI's GPT-4 models and the Swarm framework, this solution streamlines the application process while ensuring high-quality, personalized cover letters that faithfully represent candidates' qualifications.
Key challenges addressed:

- Information accuracy and verification
- Tailored content matching job requirements
- Professional standards compliance
- Automated quality improvement

## Importing Necessary Libraries

Our system relies on several Python libraries to handle different aspects of the cover letter generation process:

### Core Python Libraries
- **os**: Manages operating system interfaces and file operations
- **base64**: Handles binary-to-text encoding for image processing
- **BytesIO**: Provides in-memory binary stream capabilities for file handling

### Document Processing
- **pytesseract**: Implements Optical Character Recognition (OCR) for extracting text from images
- **pdf2image**: Converts PDF documents into processable images
- **BeautifulSoup**: Parses HTML/XML for structured data extraction and formatting

### AI and Agent Framework
- **OpenAI**: Interfaces with GPT-4 models through OpenAI's API
- **Swarm**: Implements the agent orchestration framework for coordinating multiple AI agents

These libraries form the foundation for our automated cover letter generation system, enabling seamless document processing and AI-driven content generation.

In [1]:
# Core Python libraries
import os                  # Operating system operations like file handling
import base64             # Encoding/decoding binary data for image processing
from io import BytesIO    # In-memory binary stream handling

# PDF and Image Processing
import pytesseract        # OCR (Optical Character Recognition) for text extraction
from pdf2image import convert_from_path  # Converting PDFs to images
from bs4 import BeautifulSoup  # HTML/XML parsing (if needed for formatting)

# AI and Agent Framework
from openai import OpenAI  # OpenAI API interface for GPT-4 models
from swarm import Swarm, Agent  # Agent orchestration framework

## PDF Processing and Image Conversion

We begin by implementing a PDF document processing for resume analysis. The code converts uploaded PDF resumes into TIFF images for initial processing, then transforms these images into base64-encoded JPEG strings. This conversion is crucial for compatibility with OpenAI's vision models, which require image data in a specific format for analysis. The process ensures that visual resume content can be effectively processed by our AI agents while maintaining data quality and compatibility across the system.

In [2]:
# Convert PDF to TIFF images
def pdf_to_images(pdf_path):
    return convert_from_path(pdf_path, fmt='tiff')

In [3]:
# Process PDF file
pdf_path = "sample.pdf"

# Convert images to base64 for API transmission
images = pdf_to_images(pdf_path)

images_encoded = []

for image in images:
    buffered = BytesIO()
    image.save(buffered, format="JPEG")
    img_str = base64.b64encode(buffered.getvalue()).decode('utf-8')
    images_encoded.append(img_str)
    

## Job Description Input

This section handles the job description input, a critical component for generating tailored cover letters. The system accepts multi-line text input using Python's triple quotation syntax, allowing for preservation of formatting and line breaks. This structured input ensures accurate parsing of job requirements, responsibilities, and company details that will be used by our AI agents for cover letter customization.

In [4]:
job_description = f"""Are you ready to shape the digital future of the cloud? Join the Microsoft Azure Network Manager team and be at the forefront of innovation in the world of cloud technology.

Our team provides the software systems to manage networking resources for Azure clouds. At the heart of these services is a robust software-defined networking (SDN) and network function virtualization infrastructure, designed to autonomously manage the network system. We continuously innovate to improve the agility, scalability, reliability, security, and cost-effectiveness of our services.

As a Software Engineer II on this team, you'll have the unique opportunity to architect, build, and deliver a seamless, reliable, and high-performance cloud infrastructure. This is an opportunity to be part of an exciting set of challenges and solutions in an ever-evolving landscape. We offer a flexible hybrid working style that allows you to choose where you work, be it from our state-of-the-art facilities or the comfort of your home office.

Microsoft’s mission is to empower every person and every organization on the planet to achieve more. As employees we come together with a growth mindset, innovate to empower others and collaborate to realize our shared goals. Each day we build on our values of respect, integrity, and accountability to create a culture of inclusion where everyone can thrive at work and beyond.

"up to 100% work from home in the U.S.A."

Responsibilities

Works with appropriate stakeholders to determine user requirements for a set of features.
Contributes to the identification of dependencies, and the development of design documents for a product area with little oversight.
Creates and implements code for a product, service, or feature, reusing code as applicable.
Contributes to efforts to break down larger work items into smaller work items and provides estimation.
Acts as a Designated Responsible Individual (DRI) working on-call to monitor system/product feature/service for degradation, downtime, or interruptions and gains approval to restore system/product/service for simple problems.
Remains current in skills by investing time and effort into staying abreast of current developments that will improve the availability, reliability, efficiency, observability, and performance of products while also driving consistency in monitoring and operations at scale.
Embody our Culture and Values

Qualifications

Required Qualifications:

Bachelor's Degree in Computer Science or related technical field AND 2+ years technical engineering experience with coding in languages including, but not limited to, C, C++, C#, Java, JavaScript, or Python
OR equivalent experience.
1+ years of experience with distributed systems.
Other Requirements

Ability to meet Microsoft, customer and/or government security screening requirements are required for this role. These requirements include, but are not limited to the following specialized security screenings: 
Microsoft Cloud Background Check: This position will be required to pass the Microsoft Cloud Background Check upon hire/transfer and every two years thereafter.
Preferred Qualifications

Bachelor's Degree in Computer Science
OR related technical field AND 4+ years technical engineering experience with coding in languages including, but not limited to, C, C++, C#, Java, JavaScript,
OR Python
OR Master's Degree in Computer Science or related technical field AND 2+ years technical engineering experience with coding in languages including, but not limited to, C, C++, C#, Java, JavaScript, or Python
OR equivalent experience.
Experience with cloud computing platform.
Software Engineering IC3 - The typical base pay range for this role across the U.S. is USD $98,300 - $193,200 per year. There is a different range applicable to specific work locations, within the San Francisco Bay area and New York City metropolitan area, and the base pay range for this role in those locations is USD $127,200 - $208,800 per year.

Certain roles may be eligible for benefits and other compensation. Find additional benefits and pay information here: https://careers.microsoft.com/us/en/us-corporate-pay 

Microsoft will accept applications and processes offers for these roles on an ongoing basis.

/span>

#azurecorejobs

Microsoft is an equal opportunity employer. Consistent with applicable law, all qualified applicants will receive consideration for employment without regard to age, ancestry, citizenship, color, family or medical care leave, gender identity or expression, genetic information, immigration status, marital status, medical condition, national origin, physical or mental disability, political affiliation, protected veteran or military status, race, ethnicity, religion, sex (including pregnancy), sexual orientation, or any other characteristic protected by applicable local laws, regulations and ordinances. If you need assistance and/or a reasonable accommodation due to a disability during the application process, read more about requesting accommodations."""


## Multi-Agent System Implementation with Feedback Loop

### Agent System Architecture
Three specialized AI agents work sequentially to generate and refine cover letters using the Swarm framework and OpenAI's GPT-4 models.

### Resume Reviewer Agent (Agent A)
Analyzes resume PDFs as images to extract and structure key information including personal details, skills, experience, and education. Maintains original formatting and bullet points for accurate information preservation.

### Cover Letter Writer Agent (Agent B)
Generates cover letters based on resume analysis and job requirements. Incorporates quality feedback for improvements. Features:
- Length control (250-400 words)
- Professional tone and formatting
- Information accuracy verification
- Previous feedback integration
- Job requirement alignment

### Quality Improvement Agent (Agent C)
Implements strict fact-checking with two primary focuses:
- Fact Verification (90%): Word-by-word comparison with resume
- Job Match (10%): Company/position accuracy validation

### Feedback Loop Process
1. Initial cover letter generation
2. Quality agent fact-checking
3. Feedback list generation
4. Cover letter refinement based on feedback
5. Single iteration for optimal improvement

### Implementation Features
- Direct agent communication
- Strict hallucination prevention
- Bullet-point feedback system
- Resume-based fact validation
- Exception handling and debugging

In [5]:
# Initialize OpenAI and Swarm clients
openai_client = OpenAI(
    api_key=''
    )

client = Swarm(
    client=openai_client
)

print("Starting Swarm CLI 🐝")

# Transfer functions for agent handoffs

def transfer_to_cover_letter_agent():
    print("Handing off to cover letter agent")
    return cover_letter_agent

def transfer_to_cover_letter_reviewer_agent():
    print("Handing off to cover letter reviewer agent")
    return quality_improvement_agent

# Resume reviewer agent setup and instructions

def resume_reviewer_instructions(context_variables):
    # Detailed instructions for CV analysis and information extraction
    return f"""You are a personalized resume reviewer tasked with analyzing and extracting information from a person's CV (provided as images). Your goal is to carefully review each line of the CV and organize the extracted information in a clear, structured format.
                Instructions:

                Focus Areas for Extraction:

                Personal Information:
                - Name
                - Profession
                - Contact Details (phone number, address, email)
                - Links (LinkedIn, GitHub, personal website, etc.)

                Skills:
                If multiple skillsets are listed, group them separately based on categories or themes.
                Experience:
                For each role, include:
                - Company Name
                - Position/Title
                - Start Date and End Date
                - List of Duties/Responsibilities (retain the original bullet points).
                If there are multiple experiences, group them individually.

                Education:
                Include:
                - University/Institution Name
                - Subject or Field of Study
                - CGPA (if mentioned).

                Achievements:
                - Highlight specific accomplishments noted in the CV.

                Projects:
                Extract project details, ensuring clarity in descriptions.

                Output Requirements:

                Generate a structured text document containing all extracted information.
                Use your own words when summarizing details where necessary, but do not alter bullet points under the "Experience" section.
                Maintain a professional and organized format.
                Don't add anything other than the parsed CV in the output.


                Objective:
                The goal is to create a clean, well-organized text document that accurately reflects the information from the CV, ready for further use by a cover letter writing agent."""
                # Instructions for resume parsing

resume_reviewer_agent = Agent(
    model="gpt-4o",
    name = "Resume Reviewer",
    functions = [transfer_to_cover_letter_agent],
    instructions = resume_reviewer_instructions
)

# Cover letter writer agent setup

def cover_letter_agent_instructions(context_variables):
    job_description = context_variables["job_description"]
    feedback = "No previous feedback provided."
    if 'feedback' in context_variables:
        feedback = context_variables["feedback"]
    
    previous_cover_letter = "No previously generated cover letter"
    if 'previous_cover_letter' in context_variables:
        previous_cover_letter = context_variables["previous_cover_letter"]
    
    return f"""You are a professional cover letter writer. You are provided with complete information about an applicant and a job description for a position in a company.

                Details about the job description: {job_description}
                
                Previous cover letter generated by you: {previous_cover_letter}
                
                Feedbacks from a reviewer: {feedback}

                Instructions:

                With Feedback and Cover letter Updates:
                - If feedback from other agents and a previously generated cover letter are provided, update the cover letter based on the feedback to craft the cover letter.

                Without Feedback:
                - If feedback is not provided, use the provided CV and the job description to write the cover letter.

                Requirements:
                - Omit the applicant's phone number and email address using the CV information provided by the "Resume Reviewer" agent.
                - If there's a requirement needed in the job description, but the CV information doesn't have it: skip the requirement and don't put any information related to that in the cover letter.
                - Include the company name from the job description directly in the cover letter to ensure no placeholders.
                - Craft a professional, tailored cover letter highlighting the applicant's experience, skills, and leadership qualities.
                - Ensure the cover letter is between 250 and 400 words long.
                - Use proper formatting and write in a natural, human-like tone without compromising quality.
                - Do not output anything other than the completed cover letter in the response. 

                Objective:
                - The goal is to produce a polished, professional cover letter that effectively demonstrates the applicant's suitability for the role, aligning their profile with the job requirements in a way that feels authentic and engaging.
                - No place holders for filling up later.
                - Don't have the date in the cover letter
                - No output other than the cover letter in the start or in the end.
                - Just address the Hiring Manager by Dear Hiring Manager
                - For company name, take the company name from the job description"""


cover_letter_agent = Agent(
    model="gpt-4o",
    name="Cover letter writer",
    instructions = cover_letter_agent_instructions,
    function=[transfer_to_cover_letter_reviewer_agent]
)

# Iterative improvement function
def transfer_to_quality_agent():
    print("Handing off to quality improvement agent")
    return quality_improvement_agent


    
def quality_agent_instructions(context_variables):
    resume_content = context_variables.get("resume_content", "No resume provided")
    job_description = context_variables.get("job_description", "No job description provided")
    cover_letter = context_variables.get("cover_letter", "No cover letter provided")
    
    return f"""You are a fact-checking cover letter reviewer. Your primary task is preventing hallucinations and ensuring factual accuracy.

                Context:
                RESUME: {resume_content}
                JOB_DESCRIPTION: {job_description}
                COVER_LETTER: {cover_letter}

                Verification Process:
                1. Strict Fact Check (90%):
                   - Compare each statement against resume word-by-word
                   - Reject ANY claim not explicitly in resume
                   - Flag ALL unsupported/exaggerated statements
                   
                2. Job Match (10%):
                   - Verify company/position accuracy
                   - Only include skills listed in resume
                   

                Give a list of feedbacks as bullets points to fix the generated cover letter.
                Don't output anything other than the feedbacks.
                If there is no wrong information, just print "no feedback"
                """

quality_improvement_agent = Agent(
    model="gpt-4o",
    name="Quality Fact Checker",
    instructions=quality_agent_instructions,
    functions=[transfer_to_cover_letter_agent]
)

# Iterative improvement function
def improve_cover_letter(f_messages, cover_letter_response, resume_content, job_description, client, max_iterations=1):
    messages = f_messages.copy()
    current_letter = cover_letter_response.messages[-1]["content"]
    best_version = current_letter
    # best_score = 0
    
    for iteration in range(max_iterations):
        print(f"\nQuality Improvement Iteration {iteration + 1}...")
        
        quality_response = client.run(
            agent=quality_improvement_agent,
            messages=messages,
            context_variables={
                "resume_content": resume_content,
                "job_description": job_description,
                "cover_letter": current_letter
            },
            stream=False,
            debug=False
        )
        
        feedback = quality_response.messages[-1]["content"]

        print(feedback)

        messages.extend(quality_response.messages)

        cover_letter_response_2 = client.run(
            agent=cover_letter_agent,
            messages=messages,
            context_variables={"job_description": job_description, "previous_cover_letter": current_letter, "feedback": feedback },
            stream=False,
            debug=False
        )

        best_version = cover_letter_response_2.messages[-1]["content"]

    
    return best_version


starting_agent = resume_reviewer_agent

starting_messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image_url", 
                "image_url": {
                    "url": f"data:image/jpeg;base64,{image}",
                    "detail": "low"
                }
            } for image in images_encoded
        ]
    }
]

# Phase 1: Resume Review
print("\nPhase 1: Analyzing Resume...")
messages = starting_messages.copy()

response = client.run(
    agent=starting_agent,
    messages=starting_messages,
    stream=False,
    debug=False,
)

# Debug the response structure
print("\nDEBUG - Resume Response Structure:")
print("Response type:", type(response))
print("Response messages type:", type(response.messages))
print("Last message type:", type(response.messages[-1]))

messages.extend(response.messages)
next_agent = response.agent

# Phase 2: Generate Cover Letter
print("\nPhase 2: Generating Cover Letter...")
try:
    cover_letter_response = client.run(
        agent=next_agent,
        messages=messages,
        context_variables={"job_description": job_description},
        stream=False,
        debug=False,
    )

    messages.extend(cover_letter_response.messages)
    
    # Access the content correctly
    if cover_letter_response.messages and len(cover_letter_response.messages) > 0:
        final_content = cover_letter_response.messages[-1]["content"]
        print("\nFinal Cover Letter:")
        print(final_content)
    else:
        print("Error: No messages in cover letter response")
        
except Exception as e:
    print(f"Error generating cover letter: {str(e)}")
    print("Response structure:", cover_letter_response)



# After generating the initial cover letter
print("\nPhase 3: Quality Improvement...")
final_cover_letter = improve_cover_letter(
    f_messages=messages,
    cover_letter_response=cover_letter_response,
    resume_content=response.messages[-1]["content"],  # Resume content from first agent
    job_description=job_description,
    client=client
)

print("\nFinal Improved Cover Letter:")
print(final_cover_letter)

Starting Swarm CLI 🐝

Phase 1: Analyzing Resume...

DEBUG - Resume Response Structure:
Response type: <class 'swarm.types.Response'>
Response messages type: <class 'list'>
Last message type: <class 'dict'>

Phase 2: Generating Cover Letter...
Handing off to cover letter agent

Final Cover Letter:
Dear Hiring Manager,

I am eager to apply for the Software Engineer II position with the Microsoft Azure Network Manager team. With a solid foundation in software engineering, particularly in cloud-based environments, I am excited about the opportunity to contribute to your team’s innovative work in cloud technology.

In my current role at ShopUp as a Software Development Engineer II, I have honed my skills in designing, implementing, and deploying scalable and efficient data solutions using GKE architecture. My experience in migrating microservices from AWS EC2 to GCP GKE has refined my approach to optimizing system performance and reducing operational costs. These experiences align well with